In [ ]:
%load_ext autoreload
%autoreload 2

# Import necessary libraries

In [ ]:
from pathlib import Path
import os
import pandas as pd
from collections import Counter, deque
import time
import json
import zipfile
from codecarbon import EmissionsTracker
from pathlib import Path


# Read the data

In [ ]:
# Determine the absolute project root
PROJ_ROOT = Path.cwd().parent 

# Define the interim data directory
INTERIM_DIR = PROJ_ROOT / "data" / "interim"

In [ ]:
behaviors_train = pd.read_parquet(INTERIM_DIR / "behaviors_train.parquet")
news_train = pd.read_parquet(INTERIM_DIR / "behaviors_train.parquet")

behaviors_val = pd.read_parquet(INTERIM_DIR / "behaviors_val.parquet")
news_val = pd.read_parquet(INTERIM_DIR / "news_val.parquet")

behaviors_test = pd.read_parquet(INTERIM_DIR / "behaviors_test.parquet")
news_test = pd.read_parquet(INTERIM_DIR / "news_test.parquet")

behaviors_train_val = pd.read_parquet(INTERIM_DIR / "behaviors_train_val.parquet")
news_train_val = pd.read_parquet(INTERIM_DIR / "news_train_val.parquet")

# Setup Carbon Emissions Tracking

In [ ]:
from news_recommender_system.carbon_tracking import CarbonTracker
tracker = CarbonTracker(project_name="baseline")
tracker.start_tracking()

# Implement baseline model

In [ ]:
# Define rolling window duration: 24 hours in seconds
TIME_WINDOW = 86400

# Dictionary to store clicked news articles with associated timestamps
news_clicks = {}

def update_news_clicks(current_time, past_clicked_articles):
    """
    Update the news_clicks dictionary by removing outdated clicks and adding new ones.
    """
    # Step 1: Remove old entries beyond the 24h time window
    for news_id in list(news_clicks.keys()):
        # Remove oldest click timestamps that fall outside the 24h window
        while news_clicks[news_id] and news_clicks[news_id][0] < current_time - TIME_WINDOW:
            news_clicks[news_id].popleft()
        # Remove the entry if no clicks remain for this news_id
        if not news_clicks[news_id]:
            del news_clicks[news_id]

    # Step 2: Add new clicks from the previous impression
    if past_clicked_articles:
        for news_id in past_clicked_articles[0]:
            if news_id not in news_clicks:
                news_clicks[news_id] = deque()
            news_clicks[news_id].append(past_clicked_articles[1])  # Append the timestamp of the click


def rank_news(user_impressions, current_time, past_clicked_articles):
    """
    Rank news articles in the current impression based on the number of clicks 
    in the last 24 hours.
    """
    # Update the global click stats with the current timestamp and past clicks
    update_news_clicks(current_time, past_clicked_articles)

    news_rank = []
    for news_id in user_impressions:
        if news_id in news_clicks:
            # Use the number of clicks in the past 24h
            news_rank.append((news_id, len(news_clicks[news_id])))
        else:
            # If no clicks, assign score of 0
            news_rank.append((news_id, 0))

    # Sort the news items by click count in descending order
    news_rank.sort(key=lambda x: x[1], reverse=True)

    # Return only the ordered list of news IDs
    return [news_id for news_id, _ in news_rank]


def rank_submission_format(user_impressions, current_time, past_clicked_articles):
    """
    Return the ranking positions for each news article in the user impression list.
    """
    ranked_news = rank_news(user_impressions, current_time, past_clicked_articles)
    return [ranked_news.index(news_id) + 1 for news_id in user_impressions]


def generate_prediction_file(behaviors_df, output_file="prediction.txt"):
    """
    Generate a prediction file with click-based news rankings for each impression.
    """
    past_clicked_articles = []  # Stores clicked articles from the previous row

    with open(output_file, "w") as f:
        for _, row in behaviors_df.iterrows():
            impression_id = row['ImpressionId']
            current_time = row['Timestamp']

            # Extract only news IDs (without click label)
            user_impressions = [news.split("-")[0] for news in row['Impressions'].split()]

            # Compute ranking positions based on past 24h click data
            ranked_positions = rank_submission_format(user_impressions, current_time, past_clicked_articles)

            # Write results to file in required format
            f.write(f"{impression_id} {json.dumps(ranked_positions)}\n")

            # Prepare click data from this row for use in the next iteration
            past_clicked_articles = (
                [news.split("-")[0] for news in row['Impressions'].split() if news.split("-")[1] == '1'],
                current_time
            )

    print(f"✅ Prediction file '{output_file}' successfully created.")


# Generate predictions for the validation set using the click-based popularity model
generate_prediction_file(behaviors_val, output_file="prediction_val_baseline.txt")


# Carbon Emissions Report

In [ ]:
tracker.end_report()

# Generate a truth file for evaluation

In [ ]:
# Generate ground truth file for validation set
def generate_truth_file(validation_impressions, output_file="truth.txt"):
    """
    Generates a truth.txt file with ground truth click labels.
    """
    with open(output_file, "w") as f:
        for impression_id, news_list in validation_impressions.items():
            labels = [int(news.split("-")[1]) for news in news_list]  # Click labels
            f.write(f"{impression_id} {json.dumps(labels)}\n")  # Format output

    print(f"✅ Truth file '{output_file}' successfully created.")

generate_truth_file(behaviors_val.set_index('ImpressionId')['Impressions'].apply(lambda x: x.split()), output_file="truth_val.txt")